### 1. Load CATPCHA images and get the transformed (X,y) data

In [ ]:
import torch

from helper_functions.data_transformation import get_transformed_data_v2

train_folder = "../data/train/combine"
test_folder = "../data/test/combine"
cache_file = "../data/prepared_data_cnn_v2.pth"

try:
    # Load the prepared data from cache if it already exists
    X_train, y_train, X_test, y_test = torch.load(cache_file)
    print("Prepared data loaded from cache.")

except FileNotFoundError:
    # Else prepare the data and save it
    X_train, y_train = get_transformed_data_v2(train_folder, is_train=True)
    X_test, y_test = get_transformed_data_v2(test_folder, is_train=False)
    torch.save((X_train, y_train, X_test, y_test), cache_file)
    print("Prepared data saved to cache.")


Prepared data loaded from cache.


### 2. Instantiate basic CNN model and train it

In [ ]:
from helper_functions.models.cnn_v2 import CNN, CNNParams
from helper_functions.data_transformation import IMG_HEIGHT, IMG_WIDTH

cnn_params: CNNParams = {
    "num_classes": 36,
    "input_shape": (1, IMG_HEIGHT, IMG_WIDTH), # (channels, height, width)
    "learning_rate": 1.1*1e-3,
    "num_epochs": 100,
    "batch_size": 1000,
}

cnn = CNN(cnn_params)
cnn.train_model(X_train, y_train)
torch.save(cnn)

character_performance = cnn.evaluate_character_performance(X_test, y_test)
print(character_performance)
# print("F1 Score (macro): {0:.2f}".format(f1_score(y_test, y_pred, average='macro'))


Epoch 1/100, Loss: 3.4982802867889404
Epoch 2/100, Loss: 3.273711919784546
Epoch 3/100, Loss: 3.1655123233795166
Epoch 4/100, Loss: 3.118616819381714
Epoch 5/100, Loss: 3.092092990875244
Epoch 6/100, Loss: 3.0676965713500977
Epoch 7/100, Loss: 3.037986993789673
Epoch 8/100, Loss: 3.0110995769500732
Epoch 9/100, Loss: 2.9903995990753174
Epoch 10/100, Loss: 2.9781715869903564
Epoch 11/100, Loss: 2.969245195388794
Epoch 12/100, Loss: 2.9577560424804688
Epoch 13/100, Loss: 2.951892614364624
Epoch 14/100, Loss: 2.9403748512268066
Epoch 15/100, Loss: 2.9300918579101562
Epoch 16/100, Loss: 2.918457269668579
Epoch 17/100, Loss: 2.914971113204956
Epoch 18/100, Loss: 2.9116051197052
Epoch 19/100, Loss: 2.9067227840423584
Epoch 20/100, Loss: 2.900538206100464
Epoch 21/100, Loss: 2.8964920043945312
Epoch 22/100, Loss: 2.8932082653045654
Epoch 23/100, Loss: 2.88936185836792
Epoch 24/100, Loss: 2.8869900703430176
Epoch 25/100, Loss: 2.883516550064087
Epoch 26/100, Loss: 2.8795738220214844
Epoch 27/1